
# Data Generation Demo

This notebook demonstrates the synthetic medical data generation process for the FHE NLP project.

## Overview
- Import and use the `generate_data.py` module
- Generate 10 sample medical records
- Display data in a formatted pandas DataFrame
- Visualize distributions of diseases and ages

---

In [ ]:
# Import required libraries
import sys
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Add the src directory to Python path
sys.path.append('../src')

# Import our data generation module
from data_generation.generate_data import MedicalDataGenerator

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All imports successful!")

## 1. Initialize Data Generator

Create an instance of the MedicalDataGenerator and configure it for demonstration.

In [ ]:
# Initialize the medical data generator
generator = MedicalDataGenerator()

print("🏥 Medical Data Generator initialized")
print(f"📊 Available conditions: {len(generator.medical_conditions)} types")
print(f"💊 Available medications: {len(generator.medications)} types")
print(f"🔬 Available procedures: {len(generator.procedures)} types")

## 2. Generate Sample Records

Generate 10 synthetic medical records inline for demonstration.

In [ ]:
# Generate 10 sample records
num_records = 10
print(f"🔄 Generating {num_records} synthetic medical records...\n")

# Generate the records
records = generator.generate_records(num_records)

print(f"✅ Successfully generated {len(records)} records")
print(f"📝 Each record contains {len(records[0])} fields")

# Display the first record as an example
print("\n📋 Sample Record (formatted):")
print(json.dumps(records[0], indent=2, default=str))

## 3. Display Data in Pandas DataFrame

Convert the generated records to a pandas DataFrame for better visualization and analysis.

In [ ]:
# Convert records to pandas DataFrame
df = pd.DataFrame(records)

print(f"📊 DataFrame created with shape: {df.shape}")
print(f"📋 Columns: {list(df.columns)}")

# Display basic info about the DataFrame
print("\n📈 DataFrame Info:")
print(df.info())

# Display the first few rows
print("\n🔍 First 5 records:")
display(df.head())

In [ ]:
# Display key columns in a more readable format
display_columns = ['patient_id', 'first_name', 'last_name', 'age', 'gender', 'primary_condition', 'admission_date']

print("👥 Patient Summary:")
summary_df = df[display_columns].copy()
summary_df['admission_date'] = pd.to_datetime(summary_df['admission_date']).dt.strftime('%Y-%m-%d')

# Display the formatted summary
display(summary_df)

## 4. Data Analysis and Statistics

Analyze the generated data to understand distributions and patterns.

In [ ]:
# Basic statistics
print("📊 Basic Statistics:")
print(f"👥 Total patients: {len(df)}")
print(f"🎂 Age range: {df['age'].min()} - {df['age'].max()} years")
print(f"📊 Average age: {df['age'].mean():.1f} years")

# Gender distribution
print("\n👫 Gender Distribution:")
gender_counts = df['gender'].value_counts()
for gender, count in gender_counts.items():
    percentage = (count / len(df)) * 100
    print(f"   {gender}: {count} ({percentage:.1f}%)")

# Primary conditions distribution
print("\n🏥 Primary Conditions:")
condition_counts = df['primary_condition'].value_counts()
for condition, count in condition_counts.items():
    percentage = (count / len(df)) * 100
    print(f"   {condition}: {count} ({percentage:.1f}%)")

## 5. Data Visualization

Create visualizations to show the distribution of diseases and ages using matplotlib.

In [ ]:
# Create a figure with subplots for disease and age distributions
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Medical Data Generation Demo - Data Distributions', fontsize=16, fontweight='bold')

# 1. Age Distribution
ax1.hist(df['age'], bins=8, alpha=0.7, color='skyblue', edgecolor='black')
ax1.set_title('Age Distribution', fontweight='bold')
ax1.set_xlabel('Age (years)')
ax1.set_ylabel('Frequency')
ax1.grid(True, alpha=0.3)
ax1.axvline(df['age'].mean(), color='red', linestyle='--', alpha=0.8, label=f'Mean: {df["age"].mean():.1f}')
ax1.legend()

# 2. Primary Conditions Distribution
condition_counts = df['primary_condition'].value_counts()
bars = ax2.bar(range(len(condition_counts)), condition_counts.values, 
               color=sns.color_palette("husl", len(condition_counts)), alpha=0.8)
ax2.set_title('Primary Conditions Distribution', fontweight='bold')
ax2.set_xlabel('Medical Conditions')
ax2.set_ylabel('Number of Patients')
ax2.set_xticks(range(len(condition_counts)))
ax2.set_xticklabels(condition_counts.index, rotation=45, ha='right')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, value in zip(bars, condition_counts.values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05, 
             str(value), ha='center', va='bottom', fontweight='bold')

# 3. Gender Distribution (Pie Chart)
gender_counts = df['gender'].value_counts()
colors = ['lightcoral', 'lightskyblue', 'lightgreen']
wedges, texts, autotexts = ax3.pie(gender_counts.values, labels=gender_counts.index, 
                                   autopct='%1.1f%%', colors=colors[:len(gender_counts)],
                                   startangle=90)
ax3.set_title('Gender Distribution', fontweight='bold')

# Enhance pie chart appearance
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

# 4. Age vs Primary Condition (Box Plot)
conditions = df['primary_condition'].unique()
age_by_condition = [df[df['primary_condition'] == condition]['age'].values for condition in conditions]

box_plot = ax4.boxplot(age_by_condition, labels=conditions, patch_artist=True)
ax4.set_title('Age Distribution by Primary Condition', fontweight='bold')
ax4.set_xlabel('Primary Condition')
ax4.set_ylabel('Age (years)')
ax4.tick_params(axis='x', rotation=45)
ax4.grid(True, alpha=0.3)

# Color the box plots
colors = sns.color_palette("husl", len(conditions))
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

plt.tight_layout()
plt.show()

print("📈 Visualizations completed!")

## 6. Summary and Export

Summary of the data generation demonstration and export the generated data.

In [ ]:
# Export generated data
output_dir = Path('../data/raw')
output_dir.mkdir(parents=True, exist_ok=True)

# Save as JSON
json_output_path = output_dir / 'demo_medical_records.json'
with open(json_output_path, 'w', encoding='utf-8') as f:
    json.dump(records, f, indent=2, default=str, ensure_ascii=False)

print(f"💾 Data saved to: {json_output_path}")

# Save as CSV for easy viewing
csv_output_path = output_dir / 'demo_medical_records.csv'
df.to_csv(csv_output_path, index=False)
print(f"📊 CSV saved to: {csv_output_path}")

# Summary
print("\n🎉 Data Generation Demo Summary")
print("=" * 50)
print(f"✅ Successfully generated {len(records)} synthetic medical records")
print(f"📊 Data contains {len(df.columns)} fields per record")
print(f"🏥 {len(df['primary_condition'].unique())} unique medical conditions")
print(f"👥 Age range: {df['age'].min()}-{df['age'].max()} years")
print(f"📈 Created 4 visualizations showing data distributions")
print(f"💾 Exported data in JSON and CSV formats")

print("\n🔄 Next Steps:")
print("1. 🔧 Run data preprocessing (preprocess_data.py)")
print("2. 🤖 Train FHE models (Logistic Regression, SVM)")
print("3. 📊 Evaluate model performance and privacy metrics")
print("4. 📈 Generate comprehensive visualizations")

print("\n✨ Demo completed successfully!")